# Feature Engineering

As mentioned in the end of the last notebook, we want to enrich our dataset (which currently consists only of single-gameweek data) by adding time-based cumulative features which can provide greater insight regarding player form.

Before we begin, let's look at the features we currently have:

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
# current features
processed_24_25 = pd.read_csv('../data/processed/2024-25/processed_merged_gws.csv')
current_features = processed_24_25.columns

current_features

Index(['element', 'name', 'position', 'GW', 'total_points', 'value', 'minutes',
       'expected_goals', 'expected_assists', 'expected_goals_conceded',
       'goals_scored', 'assists', 'goals_conceded', 'clean_sheets',
       'ict_index', 'fixture', 'was_home', 'fixture_difficulty'],
      dtype='object')

Now, let's identify the features that we can potentially engineer and enhance:
- **total_points**: this is the player's individual gameweek score. We can enhance this feature by taking the player's total or average gameweek score over the past few weeks.
- We can do the same for **minutes played**, **goals and assists**, **expected goals and assists**, **goals conceded**, **expected goals conceded**, and **clean sheets**.

We will choose a sliding time windows of **1 gameweek** and **3 gameweeks** as an initial attempt.

## Features to be added

- **avg_score_last_x**: Average gameweek score over the last (1,3) gameweeks.

- **avg_mins_last_x**: Average minutes played over the last (1,3) gameweeks.

- **goals_last_x**: Total goals scored over the last (1,3) gameweeks.

- **assists_last_x**: Total assists over the last (1,3) gameweeks.

- **xG_last_x**: Total expected goals over the last (1,3) gameweeks.

- **xA_last_x**: Total expected assists over the last (1,3) gameweeks.

- **ict_last_x**: Total ICT index over the last (1,3) gameweeks.

- **goals_conceded_last_x**: Total goals conceded over the last (1,3) gameweeks.

- **avg_xGC_last_x**: Average expected goals conceded over the last (1,3) gameweeks.

- **clean_sheets_last_x**: Total clean sheets over the last (1,3) gameweeks.

In [3]:
# helper function to calculate rolling features
def calculate_rolling_features(df, window, min_p):
    grouped = df.groupby('element')

    # roll statistics
    df[f'avg_score_last_{window}'] = (
        grouped['total_points'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).mean()
        ).reset_index(level=0, drop=True)
    )

    df[f'avg_mins_last_{window}'] = (
        grouped['minutes'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).mean()
        ).reset_index(level=0, drop=True)
    )

    df[f'goals_last_{window}'] = (
        grouped['goals_scored'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).sum()
        ).reset_index(level=0, drop=True)
    )

    df[f'assists_last_{window}'] = (
        grouped['assists'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).sum()
        ).reset_index(level=0, drop=True)
    )

    df[f'xG_last_{window}'] = (
        grouped['expected_goals'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).sum()
        ).reset_index(level=0, drop=True)
    )

    df[f'xA_last_{window}'] = (
        grouped['expected_assists'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).sum()
        ).reset_index(level=0, drop=True)
    )

    df[f'ict_last_{window}'] = (
        grouped['ict_index'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).sum()
        ).reset_index(level=0, drop=True)
    )

    df[f'goals_conceded_last_{window}'] = (
        grouped['goals_conceded'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).sum()
        ).reset_index(level=0, drop=True)
    )

    df[f'avg_xGC_last_{window}'] = (
        grouped['expected_goals_conceded'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).mean()
        ).reset_index(level=0, drop=True)
    )

    df[f'clean_sheets_last_{window}'] = (
        grouped['clean_sheets'].apply(
            lambda x: x.shift(1).rolling(window, min_periods=min_p).sum()
        ).reset_index(level=0, drop=True)
    )

    return df

In [4]:
# general function to faciliate feature engineering process
def feature_engineering(df):
    # sort by player and GW to ensure proper rolling calculations
    df = df.sort_values(by=['element', 'GW'])

    # main rolling calculations
    fe_df = calculate_rolling_features(df, window=3, min_p=2)
    fe_df = calculate_rolling_features(df, window=1, min_p=1)

    # drop invalid rows (with NaN values)
    fe_df = fe_df.dropna()
    
    return fe_df

### Testing

In [5]:
# test cases
def test_feature_engineering():
    # create a random dataframe
    np.random.seed(0)
    test_df = pd.DataFrame({
        'element': np.random.randint(1, 5, 20),
        'GW': np.arange(1, 21),
        'total_points': np.random.randint(0, 10, 20),
        'minutes': np.random.randint(0, 90, 20),
        'goals_scored': np.random.randint(0, 2, 20),
        'assists': np.random.randint(0, 2, 20),
        'expected_goals': np.random.random(20),
        'expected_assists': np.random.random(20),
        'ict_index': np.random.random(20),
        'goals_conceded': np.random.randint(0, 2, 20),
        'expected_goals_conceded': np.random.random(20),
        'clean_sheets': np.random.randint(0, 2, 20)
    })

    # apply feature engineering
    fe_df = feature_engineering(test_df)

    # check if new features are added
    expected_new_features = [
        'avg_score_last_3', 'avg_mins_last_3', 'goals_last_3', 'assists_last_3',
        'xG_last_3', 'xA_last_3', 'ict_last_3', 'goals_conceded_last_3', 'avg_xGC_last_3', 'clean_sheets_last_3',
        'avg_score_last_1', 'avg_mins_last_1', 'goals_last_1', 'assists_last_1',
        'xG_last_1', 'xA_last_1', 'ict_last_1', 'goals_conceded_last_1', 'avg_xGC_last_1', 'clean_sheets_last_1'
    ]
    for feature in expected_new_features:
        assert feature in fe_df.columns, f"{feature} not found in the dataframe"

    # check if no NaN values are present
    assert not fe_df.isnull().values.any(), "NaN values found in the dataframe"

    print("All tests passed!")

# run tests
test_feature_engineering()

All tests passed!


### Now, time to put everything together and enhance our data
### 2022-23

In [6]:
# load in processed data 
processed_merged_gws_22_23 = pd.read_csv('../data/processed/2022-23/processed_merged_gws.csv')

# transform
feature_engineered_22_23 = feature_engineering(processed_merged_gws_22_23)

# output
feature_engineered_22_23.to_csv('../data/processed/2022-23/feature_engineered_22_23.csv', index=False)

### 2023-24

In [7]:
# load in processed data 
processed_merged_gws_23_24 = pd.read_csv('../data/processed/2023-24/processed_merged_gws.csv')

# transform
feature_engineered_23_24 = feature_engineering(processed_merged_gws_23_24)

# output
feature_engineered_23_24.to_csv('../data/processed/2023-24/feature_engineered_23_24.csv', index=False)

### 2024-25

In [8]:
# load in processed data 
processed_merged_gws_24_25 = pd.read_csv('../data/processed/2024-25/processed_merged_gws.csv')

# transform
feature_engineered_24_25 = feature_engineering(processed_merged_gws_24_25)

print(feature_engineered_24_25)

# output
feature_engineered_24_25.to_csv('../data/processed/2024-25/feature_engineered_24_25.csv', index=False)

      element                   name position  GW  total_points  value  \
1538        1  Fábio Ferreira Vieira      MID   3             0     54   
2190        1  Fábio Ferreira Vieira      MID   4             0     54   
2849        1  Fábio Ferreira Vieira      MID   5             0     54   
3511        1  Fábio Ferreira Vieira      MID   6             0     54   
4175        1  Fábio Ferreira Vieira      MID   7             0     54   
...       ...                    ...      ...  ..           ...    ...   
5843      663          Imari Samuels      DEF   9             0     40   
4586      664             Alfie Pond      DEF   8             0     40   
5254      664             Alfie Pond      DEF   9             0     40   
5811      665  Nathan Butler-Oyedeji      FWD   9             0     45   
5781      666           Caleb Kporha      DEF   9             0     40   

      minutes  expected_goals  expected_assists  expected_goals_conceded  ...  \
1538        0             0.0 